In [1]:
import pandas as pd

In [2]:
med_data = pd.read_csv('../../data/claims_med_interns_2016-2018_20190304.csv')
med_data.columns

D:\Users\vhsu\AppData\Local\Continuum\anaconda3\lib\site-packages\IPython\core\interactiveshell.py:3049: DtypeWarning: Columns (59) have mixed types. Specify dtype option on import or set low_memory=False.
  interactivity=interactivity, compiler=compiler, result=result)


Index(['Member ID Encrypted', 'Claim ID', 'Service Line', 'Member Gender',
       'Age', 'Zip (5-digit)', 'MSA', 'County', 'State', 'Employee Type',
       'Relation', 'Line Of Business', 'Claim Status', 'Admit Type & Desc',
       'Admit Source & Desc', 'Discharge Status Code', 'Primary ICD Rollup',
       'Primary ICD Diagnosis Code', 'Primary ICD Diagnosis Desc',
       '2nd ICD Rollup', '2nd ICD Diagnosis Code', '2nd ICD Diagnosis Desc',
       '3rd ICD Rollup', '3rd ICD Diagnosis Code', '3rd ICD Diagnosis Desc',
       '4th ICD Rollup', '4th ICD Diagnosis Code', '4th ICD Diagnosis Desc',
       '5th ICD Rollup', '5th ICD Diagnosis Code', '5th ICD Diagnosis Desc',
       '6th ICD Rollup', '6th ICD Diagnosis Code', '6th ICD Diagnosis Desc',
       '7th ICD Rollup', '7th ICD Diagnosis Code', '7th ICD Diagnosis Desc',
       '8th ICD Rollup', '8th ICD Diagnosis Code', '8th ICD Diagnosis Desc',
       '9th ICD Rollup', '9th ICD Diagnosis Code', '9th ICD Diagnosis Desc',
       '10th IC

In [65]:
def reshape(group):
    df = pd.DataFrame()
    
    # Demographic info
    df['gender'] = group.head(1)['Member Gender']
    df['age'] = group.head(1)['Age']
    df['postal_code'] = group.head(1)['Zip (5-digit)']
    df['county'] = group.head(1)['County']
    
    # Lists of all codes and rollups
    icd_codes = group['Primary ICD Diagnosis Code'] + group['2nd ICD Diagnosis Code'] + group['3rd ICD Diagnosis Code'] + group['4th ICD Diagnosis Code'] + group['5th ICD Diagnosis Code'] + group['6th ICD Diagnosis Code'] + group['7th ICD Diagnosis Code'] + group['8th ICD Diagnosis Code'] + group['9th ICD Diagnosis Code'] + group['10th ICD Diagnosis Code']
    icd_rollups = group['Primary ICD Rollup'] + group['2nd ICD Rollup'] + group['3rd ICD Rollup'] + group['4th ICD Rollup'] + group['5th ICD Rollup'] + group['6th ICD Rollup'] + group['7th ICD Rollup'] + group['8th ICD Rollup'] + group['9th ICD Rollup'] + group['10th ICD Rollup']
    
    # Find if conditions exist in claims 
    hypertension = ['I10' in s for s in icd_codes]
    cardiovascular = ['heart' in s for s in icd_rollups]
    diabetes = ['Diabetes' in s for s in icd_rollups]
    obesity = ['E66' in s for s in icd_codes]
    cancer = ['Cancer' in s for s in icd_rollups]
    musculoskeletal = ['musculoskeletal' in s for s in icd_rollups]
    copd = ['Chronic obstructive pulmonary disease' in s for s in icd_rollups]
    cholesterol = ['E78' in s for s in icd_codes]

    # Set binary for conditions: 1 if condition exists, 0 otherwise
    df['binary_hypertension'] = 1 if True in hypertension else 0
    df['binary_cardiovascular'] = 1 if True in cardiovascular else 0
    df['binary_diabetes'] = 1 if True in diabetes else 0
    df['binary_obesity'] = 1 if True in obesity else 0
    df['binary_cancer'] = 1 if True in cancer else 0
    df['binary_musculoskeletal'] = 1 if True in musculoskeletal else 0
    df['binary_copd'] = 1 if True in copd else 0
    df['binary_cholesterol'] = 1 if True in cholesterol else 0
    
    # Set total number of claims for conditions
    df['claims_hypertension'] = sum(hypertension)
    df['claims_cardiovascular'] = sum(cardiovascular)
    df['claims_diabetes'] = sum(diabetes)
    df['claims_obesity'] = sum(obesity)
    df['claims_cancer'] = sum(cancer)
    df['claims_musculoskeletal'] = sum(musculoskeletal)
    df['claims_copd'] = sum(copd)
    df['claims_cholesterol'] = sum(cholesterol)
    
    # Total number of conditions and claims
    total_conditions = df['binary_hypertension'] + df['binary_cardiovascular'] + df['binary_diabetes'] + df['binary_obesity'] + df['binary_cancer'] + df['binary_musculoskeletal'] + df['binary_copd'] + df['binary_cholesterol']
    total_claims = df['claims_hypertension'] + df['claims_cardiovascular'] + df['claims_diabetes'] + df['claims_obesity'] + df['claims_cancer'] + df['claims_musculoskeletal'] + df['claims_copd'] + df['claims_cholesterol']

    # Set total number of conditions and claims
    df['total_conditions'] = total_conditions
    df['total_claims'] = total_claims
     
    # Flags for nonused (claims were not filed for our conditions) and comorbidity (has more than one condition out of the ones we are interested in)
    df['flag_nonused'] = total_claims.apply(lambda x: 1 if x == 0 else 0)
    df['flag_comorbidity'] = total_conditions.apply(lambda x: 1 if x > 1 else 0)
        
    return df

In [66]:
# Reshape data by grouping by member and year
reshaped_med_data = med_data.groupby(['Member ID Encrypted', 'Incurred Year']).apply(reshape)

In [67]:
reshaped_med_data.head(10)

gender  age  postal_code        county  \
Member ID Encrypted Incurred Year                                              
710087KT90EY0DZUV   2018          0        F   47        98335        PIERCE   
71008ADZ2VBBDM14C   2016          13       F   54        98032          KING   
7100UNBYH5SYEX19X   2016          14       F   63        98926      KITTITAS   
                    2018          21       F   65        98926      KITTITAS   
71069K607Y1CQ9BV5   2016          22       M   70        98059          KING   
                    2017          27       M   71        98059          KING   
                    2018          88       M   72        98059          KING   
7107GY7V3ZWGK7FKP   2016          246      F   57        98520  GRAYS HARBOR   
                    2017          310      F   58        98520  GRAYS HARBOR   
7107NGE7PLEN7UMDY   2016          311      F   26        98030          KING   

                                       binary_hypertension  \
Member ID Encrypted Incurred Year                            
710087KT90EY0DZUV   2018          0                      0   
71008ADZ2VBBDM14C   2016          13                     0   
7100UNBYH5SYEX19X   2016          14                     0   
                    2018          21                     0   
71069K607Y1CQ9BV5   2016          22                     0   
                    2017          27                     1   
                    2018          88                     1   
7107GY7V3ZWGK7FKP   2016          246                    1   
                    2017          310                    0   
7107NGE7PLEN7UMDY   2016          311                    0   

                                       binary_cardiovascular  binary_diabetes  \
Member ID Encrypted Incurred Year                                               
710087KT90EY0DZUV   2018          0                        0                0   
71008ADZ2VBBDM14C   2016          13                       0                0   
7100UNBYH5SYEX19X   2016          14                       0                0   
                    2018          21                       0                0   
71069K607Y1CQ9BV5   2016          22                       0                0   
                    2017          27                       1                1   
                    2018          88                       1                1   
7107GY7V3ZWGK7FKP   2016          246                      0                0   
                    2017          310                      0                0   
7107NGE7PLEN7UMDY   2016          311                      0                0   

                                       binary_obesity  binary_cancer  \
Member ID Encrypted Incurred Year                                      
710087KT90EY0DZUV   2018          0                 1              0   
71008ADZ2VBBDM14C   2016          13                0              0   
7100UNBYH5SYEX19X   2016          14                0              0   
                    2018          21                0              0   
71069K607Y1CQ9BV5   2016          22                0              0   
                    2017          27                0              0   
                    2018          88                0              0   
7107GY7V3ZWGK7FKP   2016          246               0              0   
                    2017          310               0              0   
7107NGE7PLEN7UMDY   2016          311               0              0   

                                       binary_musculoskeletal  ...  \
Member ID Encrypted Incurred Year                              ...   
710087KT90EY0DZUV   2018          0                         0  ...   
71008ADZ2VBBDM14C   2016          13                        0  ...   
7100UNBYH5SYEX19X   2016          14                        0  ...   
                    2018          21                        0  ...   
71069K607Y1CQ9BV5   2016          22                        0  ...   
    